# Uncovering Associations among Genes and Phenotypes with SEAHORSE
Author: Enakshi Saha<sup>1</sup>

<sup>1</sup>Department of Biostatistics, Harvard T. H. Chan School of Public Health, Boston, MA, USA.

# 1. Introduction

SEAHORSE (Serendipity Engine Assaying Heterogenous Omics-Related Sampling Experiments) is a tool for exploratory data analysis designed to allow users to identify both expected and unexpected associations within large data sets; SEAHORSE allows users to ask and answer questions about individual variables or cohorts. Given gene expression and clinical data from a particular cohort such as a specific tissue from the Genotype Tissue Expression (GTEx) Project or a particular cancer type from The CancerGenome Atlas (TCGA), seahorse identifies pairs of most correlated genes, and also genes and biological pathways strongly associated to any given phenotype.

You need to set the `runserver` parameter to 1, if you're running this vignette on the server. Otherwise, if the vignette is ran locally this parameter has to be set to 0.


In [ ]:
runserver=1

If you are running this vignette locally, you need to download the data set first before running the script.

Next, we need to load the SEAHORSE functions. A description of function inputs is provided below:

```
Description:
               SEAHORSE computes gene-gene coexpression matrix,
               associations between a set of given phenotypes and each gene,
               performs gene set enrichment analysis (GSEA) for each phenotype,
               using the measures of association.
               GSEA is performed through R package "fgsea".
               The measures of association 
               for a numerical phenotype is Pearson correlation and
               for a categorical phenotype is the p-value of an ANOVA test
                

Inputs:
@param expression : gene expression matrix (normalized, and filtered) 
                     with rows as genes and columns as samples.
                     Row and column names must be present.
                     Row names must be HGNC symbols.
                     Column names must match the row names of the phenotype matrix.
 @param phenotype : phenotype matrix
                    with rows as samples and columns as phenotype variables.
 @param phenotype_dictionary : a vector of strings
                               containing type of each phenotype.
                               Types can be either "numeric" or "categorical" 
 @param pathways : a list of pathways (e.g. KEGG, GO, Reactome etc. 
                   downloaded from http://www.gsea-msigdb.org/gsea/msigdb/human/collections.jsp)
 Outputs:
 @return results    : a list containing three objects
         results$coexpression: a gene x gene Pearson correlation matrix.
         results$phenotype_association : a list containing a vector for each phenotype
         results$GSEA: a list containing a matrix of GSEA results for each phenotype

 @examples

 expression_data = data.frame(matrix(rexp(200, rate=.1), ncol=10, nrow = 20))
 rownames(expression_data) = paste("gene", 1:20, sep = "")
 colnames(expression_data) = paste("sample", 1:10, sep = "")
 
 phenotype_data = data.frame(matrix(0, ncol=2, nrow = 10))
 colnames(phenotype_data) = c("sex", "height")
 rownames(phenotype_data) = colnames(expression_data)
 phenotype_data$sex = c(rep("male", nrow(phenotype_data)/2), rep("female", nrow(phenotype_data)/2))
 phenotype_data$height = 65 + sample.int(10, nrow(phenotype_data), replace = T)
 
 phenotype_dictionary = c("categorical", "numeric")
 
 pathways = list()
 pathways$pathway1 = sample(rownames(expression_data), 5)
 pathways$pathway2 = sample(rownames(expression_data), 3)
 pathways$pathway1 = sample(rownames(expression_data), 7)

# Run seahorse
results <- seahorse(expression_data, phenotype_data, phenotype_dictionary, pathways)
```

The first SEAHORSE function runs GSEA for a numeric phenotype.

In [ ]:
gsea_numeric <- function(expression, pheno, pathways, results){
  output_seahorse = list()
  output_seahorse$cor = list()
  output_seahorse$GSEA = list()
  
  phenotype_vector = as.numeric(pheno)
  cor = unlist(apply(expression, MARGIN=1, function(x){cor(as.numeric(x), phenotype_vector, use="pairwise.complete.obs")}))
  output_seahorse$cor = cor
  
  # Run GSEA
  cor_rank = sort(cor, decreasing = T)
  fgseaRes <- fgsea(pathways, cor_rank, minSize=15, maxSize=500)
  output_seahorse$GSEA = fgseaRes
  
  return(output_seahorse)
}

The following SEAHORSE function runs GSEA for a categorical phenotype.

In [ ]:
gsea_categorical <- function(expression, pheno, pathways, results){
  output_seahorse = list()
  output_seahorse$cor = list()
  output_seahorse$GSEA = list()
  
  phenotype_vector = factor(as.character(pheno))
  cor = unlist(apply(expression, MARGIN=1, function(x){anova(lm(as.numeric(x)~phenotype_vector))$`Pr(>F)`[1]}))
  output_seahorse$cor = cor
  
  # Run GSEA
  cor_rank = sort(cor, decreasing = T)
  fgseaRes <- fgsea(pathways, cor_rank, minSize=15, maxSize=500, scoreType = "pos")
  output_seahorse$GSEA = fgseaRes
  
  return(output_seahorse)
}

Finally, this is the main SEAHORSE function that puts everything together.

In [ ]:
seahorse <- function(expression, phenotype, phenotype_dictionary, pathways){
  set.seed(0)
  library(fgsea)
  
  results = list()
  
  # Compute coexpression of genes
  results$coexpression = cor(t(expression), use="pairwise.complete.obs")
  
  # Compute association of gene expression with phenotypes and run GSEA
  results$phenotype_association = list()
  results$GSEA = list()
  
  for (i in 1:ncol(phenotype)){
    pheno = phenotype[,i]
    pheno_name = colnames(phenotype)[i]
    
    if (phenotype_dictionary[i] == "numeric"){
      output_seahorse = gsea_numeric(expression, pheno, pathways, results)
    }else {output_seahorse = gsea_categorical(expression, pheno, pathways, results)}
    results$phenotype_association[[pheno_name]] = output_seahorse$cor
    results$GSEA[[pheno_name]] = output_seahorse$GSEA
  }
  return(results)
}

We also need to load the package fgsea to run gene set enrichment analysis.

In [ ]:
library(fgsea)

# 2. A Simple Example with Toy Data Generated from GTEx Lung Samples
We download TPM-normalized RNAseq data from http://gtexportal.org/ and select 200 random lung tissue samples from the GTEx (version 8) database. We choose one categorical phenotype "SEX" and one numeric phenotype "HGHT" (height) to demonstrate how SEAHORSE identifies associations from gene expression and phenotypic data.

The phenotype HGHT is a dbGap-protected and is not publicly available. To protect data privacy we add random noise to the HGHT variable to generate toy phenotypic data for each of the 200 individuals.

We filter out lowly expressed genes (genes with count < 1TPM in at least 5% of all samples).

The expression and the phenotypic data can be downloaded as below.

In [ ]:
if (runserver==0){
    system("curl -O  https://netzoo.s3.us-east-2.amazonaws.com/netzoo/netbooks/seahorse/GTEx_lung_expression_toydata.txt")
    system("curl -O  https://netzoo.s3.us-east-2.amazonaws.com/netzoo/netbooks/seahorse/GTEx_lung_phenotype_toydata.txt")
}

On the server, the expression and the phenotypic data can be loaded as follows:

In [ ]:
file_path = "/opt/data/netZooR/seahorse/"
# Load expression data
expression_data <- read.csv(paste0(file_path, "GTEx_lung_expression_toydata.txt"), sep="")
# Load phenotypic data
phenotype_data <- read.csv(paste0(file_path, "GTEx_lung_phenotype_toydata.txt"), sep="")

Let us first print the first 5 rows of the phenotypic data to see what kinds of phenotypes we have.

In [ ]:
head(phenotype_data)

We need to create a vector of same length as the number of phenotypes recording the type ("numeric" or "categorical") of each phenotypic variable. We see from above that "SEX" is a categorical variable with two categories and "HGHT" is a numeric variable.


In [ ]:
phenotype_dictionary = c("categorical", "numeric")

Next we load a list of biological pathways to perform gene set enrichment analysis for every phenotypic variable, using the association between each gene and the phenotype under consideration. Here we use the KEGG biological pathways downloaded from https://www.gsea-msigdb.org/gsea/msigdb/human/genesets.jsp?collection=H that contains 187 pathways. The pathway file can be downloaded as below:

In [ ]:
if (runserver==0){
    system("curl -O  https://netzoo.s3.us-east-2.amazonaws.com/netzoo/netZooR/tutorial_datasets/seahorse/c2.cp.kegg.v2022.1.Hs.symbols.gmt")
}

On the server, the list of pathways can be loaded as follows:

In [ ]:
# Load KEGG pathways
pathways_KEGG <- gmtPathways(paste0(file_path, "c2.cp.kegg.v2022.1.Hs.symbols.gmt"))

Finally, we run SEAHORSE using the gene expression and the phenotypic data, the phenotype dictionary vector we created and the list of pathways.

In [ ]:
results = seahorse(expression = expression_data, phenotype = phenotype_data, 
                   phenotype_dictionary, pathways = pathways_KEGG)

The SEAHORSE output contains three components: 
- the coexpression matrix containing Pearson's correlation for every pair of genes.
- the measure of association between every gene and every phenotype.
- a table containing the results from gene set enrichment analysis (GSEA) for every phenotype.

Let us examine each component one by one.

First, the gene-gene coexpression matrix can be used to identify most correlated genes. For example, suppose we are interested to find top 10 genes most strongly correlated (positive or negative correlation) with the gene "EGFR". The mutated forms of the EGFR gene have been found in some types of cancer, including non-small cell lung cancer.

In [ ]:
# Coexpression matrix
cor = results$coexpression
# Coexpression of all genes with EGFR
cor_EGFR = cor[which(rownames(cor) == "EGFR"),]
# Top 10 genes (excluding EGFR itself) most correlated with EGFR
cor_EGFR[order(abs(cor_EGFR),decreasing = T)[2:11]]

We can identify top biological pathways that are most differentially expressed between males and females in lung tissue using the association component of the SEAHORSE results. The association between a categorical phenotype and gene expression is measured by the p-value of an ANOVA test.

In [ ]:
# Correlation between height (HGHT) and all genes
cor_sex = results$phenotype_association[["SEX"]]
# Top 10 genes most differentially expressed by sex
cor_sex[order(cor_sex)][1:10]

Next, we identify top genes that are most (positively or negatively) associated with height using the association component of the SEAHORSE results. The association between a numeric phenotype and gene expression is measured by the Pearson correlation coefficient.

In [ ]:
# Correlation between height (HGHT) and all genes
cor_hght = results$phenotype_association[["HGHT"]]
# Top 10 genes most correlated with height (HGHT)
cor_hght[order(abs(cor_hght), decreasing = T)][1:10]

We can also identify the biological pathways significantly correlated (say, at a false discovery rate cutoff 0.05) with height in the lung tissue. The following code prints top 5 biological pathways most associated with height.

In [ ]:
# Gene set enrichment analysis results for height (HGHT)
GSEA_hght = results$GSEA[["HGHT"]]
# Top 5 biological pathways most associated with height.
GSEA_hght[order(GSEA_hght$padj),][1:5,]

# References

1- Ben Guebila, Marouen, Tian Wang, Camila M. Lopes-Ramos, Viola Fanfani, Des Weighill, Rebekka Burkholz, Daniel Schlauch et al. "The Network Zoo: a multilingual package for the inference and analysis of gene regulatory networks." Genome Biology 24, no. 1 (2023): 45.

2- Lonsdale, John, Jeffrey Thomas, Mike Salvatore, Rebecca Phillips, Edmund Lo, Saboor Shad, Richard Hasz et al. "The genotype-tissue expression (GTEx) project." Nature genetics 45, no. 6 (2013): 580-585.

3- Data Coordinating Center Burton Robert 67 Jensen Mark A 53 Kahn Ari 53 Pihl Todd 53 Pot David 53 Wan Yunhu 53, and Tissue Source Site Levine Douglas A 68. "The cancer genome atlas pan-cancer analysis project." Nature genetics 45, no. 10 (2013): 1113-1120.

4- Kanehisa, Minoru. "The KEGG database." In ‘In Silico’Simulation of Biological Processes: Novartis Foundation Symposium 247, vol. 247, pp. 91-103. Chichester, UK: John Wiley & Sons, Ltd, 2002.

5- Mailman, Matthew D., Michael Feolo, Yumi Jin, Masato Kimura, Kimberly Tryka, Rinat Bagoutdinov, Luning Hao et al. "The NCBI dbGaP database of genotypes and phenotypes." Nature genetics 39, no. 10 (2007): 1181-1186.